## Agentic Workflow Design Pattern - Prompt Chaining

The Routing Agentic Workflow Design Pattern is a foundational design pattern in agentic AI systems. It enables intelligent classification of user inputs and dynamic delegation to specialized agents, tools, prompts, or sub workflows.

### Core Concept
A central "router" component typically powered by a large language model (LLM) analyzes the incoming query to determine its intent, category, complexity, or type. Based on this classification, it routes the task to the most appropriate specialized handler. This creates separation of concerns, allowing each downstream component to be optimized for specific tasks without compromising overall system performance.
Key Steps

In [33]:
from pydantic import BaseModel
from typing_extensions import Literal, TypedDict
from pydantic import Field
from langgraph.graph import END, StateGraph, START
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [34]:
class RouterOutput(TypedDict):
    output: Literal['github', 'medium', 'none'] = Field(
        default='none',
        description="The output destination, either 'github' or 'medium'. If neither is suitable, use 'none'."
    ),
    user_handle: str = Field(
        default='none',
        description="The user's profile handle for the specified output destination."
    )

In [35]:
class SharedState(TypedDict):
    query: str
    user_handle: str
    destination: Literal['github', 'medium']
    profile_summary: str
    model: ChatGroq(model_name="llama-3.3-70b-versatile")

In [36]:
def build_model(shared_state: SharedState):
    shared_state['model'] = ChatGroq(model_name="llama-3.3-70b-versatile")
    return shared_state

In [37]:
def router_agent(shared_state: SharedState):
    prompt = f"""
    You are an expert social media profile assistant. Based on the user's preference, you will determine whether to pull their GitHub or Medium profile details.
    
    Ensure your response is a valid JSON object matching the RouterOutput schema.
    """

    model_with_structured_output = shared_state['model'].with_structured_output(RouterOutput)
    grade_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("human", "User's Query: \n\n {query}"),
        ]
    )

    retrieval_grader = grade_prompt | model_with_structured_output
    response = retrieval_grader.invoke({"query": shared_state['query']})
    shared_state['destination'] = response["output"]
    shared_state['user_handle'] = response["user_handle"]

    return shared_state

In [48]:
load_dotenv()
shared_state = build_model({
    "query": "Can you summarize medium profile of aadhilimam96",
})
router_agent(shared_state)
print(shared_state['destination'], shared_state['user_handle'])

medium aadhilimam96


In [39]:
# Conditional edge function to route to the appropriate node
def route_destination(shared_state: SharedState):
    # Return the node name you want to visit next
    if shared_state["destination"] == "github":
        return "summarize_github_profile"
    elif shared_state["destination"] == "medium":
        return "summarize_medium_profile"
    else:
        return "end_node"

In [40]:
# Helper Functions
def read_github_profile(user_handle: str):
    '''Reads the GitHub profile content for the given GitHub handle.'''
    print('Reading GitHub profile content...')
    github_profile_url = 'https://www.github.com/' + user_handle
    documents = WebBaseLoader(github_profile_url).load()
    page_content = ''

    for document in documents:
        page_content += document.page_content
    
    return page_content.strip()

def read_medium_profile(user_handle: str):
    '''Reads the Medium profile content for the given Medium user handle.'''
    print('Reading Medium profile content...')
    medium_profile_url = f'https://{user_handle}.medium.com'
    documents = WebBaseLoader(medium_profile_url).load()
    page_content = ''

    for document in documents:
        page_content += document.page_content
    
    return page_content.strip()

In [41]:
def summarize_github_profile(shared_state: SharedState) -> SharedState:
    '''Summarizes the GitHub profile content.'''
    print('Summarizing GitHub profile content...') 
    model = shared_state['model']
    github_profile_content = read_github_profile(shared_state['user_handle'])   

    prompt = f'''Summarize the following GitHub profile content in a concise manner
    and highlight name, organization, followers, location, contact information key skills, 
    projects, and contributions.

    Github Profile Content: {github_profile_content}
    '''
    response = model.invoke(prompt)
    shared_state['profile_summary'] = response.content.strip()

    return shared_state


In [42]:
def summarize_medium_profile(shared_state: SharedState) -> SharedState:
    '''Summarizes the Medium profile content.'''
    print('Summarizing Medium profile content...')
    model = shared_state['model']
    medium_profile_content = read_medium_profile(shared_state['user_handle'])

    prompt = f'''Summarize the following Medium profile content in a concise manner
    and highlight name, followers, posted articles, and description.

    Medium Profile Content: {medium_profile_content}
    '''
    response = model.invoke(prompt)
    shared_state['profile_summary'] = response.content.strip()

    return shared_state

In [43]:
def build_graph():
    workflow = StateGraph(SharedState)

    # Add Nodes
    workflow.add_node(build_model, "build_model")
    workflow.add_node(router_agent, "router_agent")
    workflow.add_node(summarize_github_profile, "summarize_github_profile")
    workflow.add_node(summarize_medium_profile, "summarize_medium_profile")

    workflow.add_edge(START, "build_model")
    workflow.add_edge("build_model", "router_agent")
    workflow.add_conditional_edges(
        "router_agent", 
        route_destination, 
        { 
            "summarize_github_profile": "summarize_github_profile",
            "summarize_medium_profile": "summarize_medium_profile",
            "end_node": END
        }
    )
    workflow.add_edge("summarize_github_profile", END)
    workflow.add_edge("summarize_medium_profile", END)

    return workflow.compile()

In [44]:
load_dotenv()
compiled_graph = build_graph()
workflow_response: SharedState = compiled_graph.invoke(
    {
        "query": "Summarize the medium profile for user handle aadhilimam"
    }
)

print(f"\n Profile Type: {workflow_response['destination']}")
if workflow_response['destination'] != 'none':
    print(f"\n\n Profile Summary: {workflow_response['profile_summary']}")
else:
    print("\n\n No suitable profile found.")

Summarizing Medium profile content...
Reading Medium profile content...

 Profile Type: medium


 Profile Summary: Unfortunately, there is no information provided in the Medium profile content to summarize. The content only displays a message requesting to enable JavaScript and cookies, and does not include any details about the profile.

If you could provide the actual Medium profile content, I would be happy to help you with the following:

* **Name**: 
* **Followers**: 
* **Posted Articles**: 
* **Description**: 

Please provide the actual content, and I'll be glad to assist you.
